<a href="https://colab.research.google.com/github/pachterlab/CP_2023/blob/main/notebooks/celltypePredMCML/utero_e105_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import os


from tqdm import tnrange, tqdm_notebook
def download_file(doi,ext):
	url = 'https://api.datacite.org/dois/'+doi+'/media'
	r = requests.get(url).json()
	netcdf_url = r['data'][0]['attributes']['url']
	r = requests.get(netcdf_url,stream=True)
	#Set file name
	fname = doi.split('/')[-1]+ext
	#Download file with progress bar
	if r.status_code == 403:
		print("File Unavailable")
	if 'content-length' not in r.headers:
		print("Did not get file")
	else:
		with open(fname, 'wb') as f:
			total_length = int(r.headers.get('content-length'))
			pbar = tnrange(int(total_length/1024), unit="B")
			for chunk in r.iter_content(chunk_size=1024):
				if chunk:
					pbar.update()
					f.write(chunk)
		return fname




In [2]:
#Read in files from NCBI GEO for Integrated Utero E10.5 dataset
!wget --quiet https://ftp.ncbi.nlm.nih.gov/geo/series/GSE149nnn/GSE149372/suppl/GSE149372_GSM5068636-GSM5068641_scRNA.MetaData.105.csv.gz
!wget --quiet https://ftp.ncbi.nlm.nih.gov/geo/series/GSE149nnn/GSE149372/suppl/GSE149372_GSM5068636-GSM5068641_scRNA.normalized.assay105.csv.gz
!wget --quiet https://ftp.ncbi.nlm.nih.gov/geo/series/GSE149nnn/GSE149372/suppl/GSE149372_GSM5068636-GSM5068641_scRNA.integrated.scaled.assay105.csv.gz

# !wget --quiet https://ftp.ncbi.nlm.nih.gov/geo/series/GSE149nnn/GSE149372/suppl/GSE149372_scRNA.MetaData.85.csv.gz
# !wget --quiet https://ftp.ncbi.nlm.nih.gov/geo/series/GSE149nnn/GSE149372/suppl/GSE149372_scRNA.normalized.assay85.csv.gz
#os.system("gunzip *.gz")

#integrated.scaled.assay105

!gunzip *.gz

In [3]:
%cd /content

/content


In [4]:
!git clone https://github.com/hhcho/densvis.git

Cloning into 'densvis'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 145 (delta 3), reused 2 (delta 2), pack-reused 137
Receiving objects: 100% (145/145), 419.61 KiB | 6.36 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [5]:
%cd /content/densvis/densne/

/content/densvis/densne


In [6]:
!g++ sptree.cpp densne.cpp densne_main.cpp -o den_sne -O2
import densne

densne.cpp: In function ‘bool DENSNE::load_data(double**, int*, int*, int*, double*, double*, int*, int*, double*, double*, bool*, double**)’:
densne.cpp:961:8: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  961 |   fread(n, sizeof(int), 1, h);                                            // number of datapoints
      |   ~~~~~^~~~~~~~~~~~~~~~~~~~~~
densne.cpp:962:8: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  962 |   fread(d, sizeof(int), 1, h);                                            // original dimensionality
      |   ~~~~~^~~~~~~~~~~~~~~~~~~~~~
densne.cpp:963:8: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  963 |   fread(theta, sizeof(double), 1, h);                                        // g

In [7]:

%cd /content/

/content


In [8]:
!git clone https://github.com/pachterlab/CP_2023.git

Cloning into 'CP_2023'...
remote: Enumerating objects: 905, done.
remote: Counting objects: 100% (905/905), done.
remote: Compressing objects: 100% (403/403), done.
remote: Total 905 (delta 495), reused 836 (delta 472), pack-reused 0
Receiving objects: 100% (905/905), 340.29 MiB | 24.49 MiB/s, done.
Resolving deltas: 100% (495/495), done.
Updating files: 100% (98/98), done.


In [9]:
!pip3 install --quiet torch
# !pip3 install --quiet torch==1.9.0
!pip3 install --quiet anndata
!pip3 install --quiet matplotlib
!pip3 install --quiet scikit-learn
!pip3 install --quiet torchsummary
!pip install --quiet scanpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 5.5 MB/s eta 0:00:00


In [10]:
%cd /content/CP_2023/scripts

/content/CP_2023/scripts


## **Install Packages**

In [11]:
import anndata 
import pandas as pd
import numpy as np


import tools as tl
import random

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.neighbors import NeighborhoodComponentsAnalysis, NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale
import torch
import time
import scanpy as sc
import seaborn as sns
import umap
from scipy import stats
import scipy.io as sio
sns.set_style('white')

### **Import Data**

In [12]:
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams['axes.linewidth'] = 0.1

state = 42
ndims = 2

data_path = '../..'

pcs = 50
pcs2 = 100
n_latent = 50

In [13]:
def readLgMat(fname, data_path=data_path):

  col_names = pd.read_csv(data_path+fname, nrows=0).columns
  types_dict = {'Unnamed: 0': str}
  types_dict.update({col: np.float16 for col in col_names if col not in types_dict})

  chunk = pd.read_csv(data_path+fname,chunksize=1000,index_col=0,dtype=types_dict)
  counts = pd.concat(chunk)

  return counts

In [14]:
counts = readLgMat('/GSE149372_GSM5068636-GSM5068641_scRNA.integrated.scaled.assay105.csv',data_path)
counts.head()

,AAACCCAAGCCTCACG-1,AAACCCAAGCTCGACC-1,AAACCCAAGTTAACAG-1,AAACCCACACCATATG-1,AAACCCAGTCACGCTG-1,AAACCCAGTGCGTTTA-1,AAACCCAGTGGCTCTG-1,AAACCCATCCCATAGA-1,AAACGAAAGAAGCTCG-1,AAACGAAAGACTAAGT-1,...,TTTGGAGTCCCGAGTG-12,TTTGGTTAGTCTAACC-12,TTTGGTTCACAAAGCG-12,TTTGGTTGTATGAGGC-12,TTTGGTTTCGTGGCTG-12,TTTGTTGAGCAGCGAT-12,TTTGTTGCAGTCCGTG-12,TTTGTTGCAGTTCACA-12,TTTGTTGGTCGTGTTA-12,TTTGTTGTCCAACCAA-12
Hba-a1,-0.366699,-0.303711,-0.294678,2.830078,3.193359,-0.350342,2.978516,-0.426758,-0.448486,-0.161011,...,-0.313965,-0.265381,-0.435791,-0.354004,-0.579102,-0.314209,2.933594,-0.290771,-0.413330,-0.441895
Hbb-bh1,-0.391846,-0.164429,-0.295654,2.917969,3.232422,-0.368408,2.914062,-0.369385,-0.449463,-0.322998,...,-0.771973,-0.245239,-0.029694,-0.744141,-0.196533,-0.491455,2.970703,0.188110,-0.280029,-0.370117
Hba-x,0.409912,-0.145752,-0.494385,2.945312,3.166016,-0.245972,2.966797,-0.169556,-0.375977,-0.462646,...,-0.627441,-0.453369,-0.015388,-0.108826,0.317627,-0.010666,2.923828,-0.601562,-0.346924,0.152100
Gfod2,-0.947266,-0.459229,-0.254395,-0.182373,0.062988,0.299316,0.217407,-0.358154,-0.587402,-0.140625,...,0.168213,-0.163818,3.056641,-0.401123,1.704102,-0.355225,-0.213501,-0.847656,0.135742,1.581055
Mt1,-0.296143,-0.667480,-0.426270,1.979492,1.252930,-0.436035,2.855469,0.118347,-0.274170,1.331055,...,-0.440186,-0.308594,-0.475098,-0.369385,-0.430664,-0.371094,2.318359,-0.437744,-0.575684,-0.642090


In [15]:
cellMeta = pd.read_csv(data_path+'/GSE149372_GSM5068636-GSM5068641_scRNA.MetaData.105.csv')
print(cellMeta.head())

labs = pd.Categorical(cellMeta['integrated_snn_res.0.3'])

           Unnamed: 0 orig.ident  nCount_RNA  nFeature_RNA  samples Utero  \
0  AAACCCAAGCCTCACG-1      p10.5        4451          2028        1  ExUt   
1  AAACCCAAGCTCGACC-1      p10.5        8096          2793        1  ExUt   
2  AAACCCAAGTTAACAG-1      p10.5        6994          2760        1  ExUt   
3  AAACCCACACCATATG-1      p10.5       19467          3424        1  ExUt   
4  AAACCCAGTCACGCTG-1      p10.5        8205          1481        1  ExUt   

   percent.mt  integrated_snn_res.0.3  
0    5.526848                       7  
1    4.088439                       5  
2    3.345725                       5  
3    4.166025                       3  
4    1.962218                       3  


In [16]:
#Center and scale data
count_mat = counts.values.T

adata = anndata.AnnData(X = count_mat)
adata.obs_names = list(counts.columns)
adata.var_names = list(counts.index)
adata.obs['Utero'] = pd.Categorical(cellMeta.Utero)
adata.obs['Cluster'] = pd.Categorical(cellMeta['integrated_snn_res.0.3'])
adata

AnnData object with n_obs × n_vars = 56528 × 2000
    obs: 'Utero', 'Cluster'

In [17]:
#In paper, use top 2000 HVGs and 15 PCs
adata.obsm['log'] = adata.X #Data is already scaled so 'log' here is not just log-normalized

sc.pp.scale(adata, max_value=10) #Already scaled
adata

AnnData object with n_obs × n_vars = 56528 × 2000
    obs: 'Utero', 'Cluster'
    var: 'mean', 'std'
    obsm: 'log'

In [18]:
count_mat = adata.obsm['log']
scaled_mat = adata.X

In [19]:
counts2_orig = readLgMat('/GSE149372_GSM5068636-GSM5068641_scRNA.normalized.assay105.csv')
counts2 = counts2_orig.loc[list(adata.var_names)]
counts2.shape

(2000, 56528)

In [20]:

count_mat_orig = counts2.values.T

In [21]:
adata2 = anndata.AnnData(X = count_mat_orig)
adata2.obs_names = list(counts2.columns)
adata2.var_names = list(counts2.index)
adata2.obs['Utero'] = pd.Categorical(cellMeta.Utero)
adata2.obs['Cluster'] = pd.Categorical(cellMeta['integrated_snn_res.0.3'])
adata2

AnnData object with n_obs × n_vars = 56528 × 2000
    obs: 'Utero', 'Cluster'

In [22]:
adata2.obsm['log'] = adata2.X #Data matrix is log-normalized 

adata2.raw = adata2.copy()

sc.pp.scale(adata2, max_value=10)
adata2

AnnData object with n_obs × n_vars = 56528 × 2000
    obs: 'Utero', 'Cluster'
    var: 'mean', 'std'
    obsm: 'log'

In [23]:
count_mat_orig = adata2.obsm['log']
scaled_mat_orig = adata2.X

In [24]:
def knn_infer(embd_space, labeled_idx, labeled_lab, unlabeled_idx,n_neighbors=50):
	"""
	Predicts the labels of unlabeled data in the embedded space with KNN.
	Parameters
	----------
	embd_space : ndarray (n_samples, embedding_dim)
		Each sample is described by the features in the embedded space.
		Contains all samples, both labeled and unlabeled.
	labeled_idx : list
		Indices of the labeled samples (used for training the classifier).
	labeled_lab : ndarray (n_labeled_samples)
		Labels of the labeled samples.
	unlabeled_idx : list
		Indices of the unlabeled samples.
	Returns
	-------
	pred_lab : ndarray (n_unlabeled_samples)
		Inferred labels of the unlabeled samples.
	"""

	# obtain labeled data and unlabled data from indices
	labeled_samp = embd_space[labeled_idx, :]
	unlabeled_samp = embd_space[unlabeled_idx, :]

	from sklearn.neighbors import KNeighborsClassifier

	knn = KNeighborsClassifier(n_neighbors=n_neighbors)
	knn.fit(labeled_samp, labeled_lab)

	pred_lab = knn.predict(unlabeled_samp)
	return pred_lab


In [25]:
def getJac(orig_indices,latents, latentLab, n_neighbors=30,p=1):
  emb = []

  xs = []

  ys = []

  knnDF = pd.DataFrame()

  for p in range(len(latents)):
    i = latents[p]
    l = latentLab[p]

    ind = tl.getNeighbors(i, n_neigh = n_neighbors,p=p)

    x = tl.getJaccard(orig_indices,ind)

    xs += x
    #ys += list(y)

    emb += [l]*len(x)

    print(l)

    print(np.mean(tl.getJaccard(orig_indices,ind)))



  knnDF['x'] = xs
  #knnDF['y'] = ys
  knnDF['latent'] = emb

  return knnDF

### **Prediction Accuracy for Cell Type Labels Across Benchmarks**
Tests accuracy on sex labels as well


Set up metadata for MCML

In [26]:
lab4 = list(cellMeta['integrated_snn_res.0.3'])


First test 2D space predictions (t-SNE, UMAP, UMAP-Supervised)

In [27]:
adata = adata2

In [28]:
ndims = 2
acc_score_2D = []

for i in range(3):
  reducer = umap.UMAP(n_components = ndims)
  tsne = TSNE(n_components = ndims) 


  tsvd = TruncatedSVD(n_components=pcs)
  x_pca = tsvd.fit_transform(scaled_mat_orig)

  pcaUMAP = reducer.fit_transform(x_pca)
  pcaTSNE = tsne.fit_transform(x_pca)

  #Partially labeled UMAP

  labels = np.array([lab4]).copy().astype(np.int8)
  train_inds = np.random.choice(len(scaled_mat_orig), size = int(0.7*len(scaled_mat_orig)),replace=False) #0.7 for training fraction
  #Set 30% to no label (nan)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = -1

  pcaUMAPLab = reducer.fit_transform(x_pca,y=labels[0])

  preds = knn_infer(pcaUMAPLab, train_inds, adata.obs['Cluster'].values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs['Cluster'].values[unlab_inds], preds)
  acc_score_2D.append(acc)



  preds = knn_infer(pcaUMAP, train_inds, adata.obs['Cluster'].values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs['Cluster'].values[unlab_inds], preds)
  acc_score_2D.append(acc)

  preds = knn_infer(pcaTSNE, train_inds, adata.obs['Cluster'].values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs['Cluster'].values[unlab_inds], preds)
  acc_score_2D.append(acc)

In [31]:
print(acc_score_2D)

[0.8889085441358571, 0.8488708060616782, 0.8851937024588714, 0.8810071348546494, 0.8504039153251961, 0.8842502506043989, 0.8928002830355564, 0.8590718792381626, 0.8879650922813845]


In [28]:
acc_score_2D = [0.8889085441358571, 0.8488708060616782, 0.8851937024588714, 0.8810071348546494, 0.8504039153251961, 0.8842502506043989, 0.8928002830355564, 0.8590718792381626, 0.8879650922813845]

In [29]:
ndims = 2
acc_score_15_2D = []

for i in range(3):
  reducer = umap.UMAP(n_components = ndims)
  tsne = TSNE(n_components = ndims) 


  tsvd = TruncatedSVD(n_components=15)
  x_pca = tsvd.fit_transform(scaled_mat_orig)

  pcaUMAP = reducer.fit_transform(x_pca)
  pcaTSNE = tsne.fit_transform(x_pca)

  #Partially labeled UMAP

  labels = np.array([lab4]).copy().astype(np.int8)
  train_inds = np.random.choice(len(scaled_mat_orig), size = int(0.7*len(scaled_mat_orig)),replace=False) #0.7 for training fraction
  #Set 30% to no label (nan)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = -1

  pcaUMAPLab = reducer.fit_transform(x_pca,y=labels[0])

  preds = knn_infer(pcaUMAPLab, train_inds, adata.obs['Cluster'].values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs['Cluster'].values[unlab_inds], preds)
  acc_score_15_2D.append(acc)



  preds = knn_infer(pcaUMAP, train_inds, adata.obs['Cluster'].values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs['Cluster'].values[unlab_inds], preds)
  acc_score_15_2D.append(acc)

  preds = knn_infer(pcaTSNE, train_inds, adata.obs['Cluster'].values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs['Cluster'].values[unlab_inds], preds)
  acc_score_15_2D.append(acc)

In [30]:
print(acc_score_15_2D)

[0.8748746978005779, 0.8377262810307211, 0.8703933014918332, 0.8723981366825874, 0.8592487764608763, 0.8754053894687187, 0.8718084792735421, 0.8481042514299192, 0.8717495135326375]


PCA 50D

In [33]:

acc_scorePCA = []


for i in range(3):

  tsvd = TruncatedSVD(n_components=pcs)
  x_pca = tsvd.fit_transform(scaled_mat_orig)
  
  labels = np.array([lab4]).copy().astype(np.int8)
  train_inds = np.random.choice(len(scaled_mat_orig), size = int(0.7*len(scaled_mat_orig)),replace=False) #0.7 for training fraction
  #Set 30% to no label (nan)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = -1
  
  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs['Cluster'].values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs['Cluster'].values[unlabeled_idx], preds)
  acc_scorePCA.append(acc)

 
  # print(f"nnNCA fit in {toc - tic:0.4f} seconds")

In [34]:
print(acc_scorePCA)

[0.9053599858482222, 0.9072468895571673, 0.9040627395483224]


PCA 100D

In [35]:

acc_scorePCA100D = []

for i in range(3):

  tsvd = TruncatedSVD(n_components=100)
  x_pca = tsvd.fit_transform(scaled_mat_orig)
  
  labels = np.array([lab4]).copy().astype(np.int8)
  train_inds = np.random.choice(len(scaled_mat_orig), size = int(0.7*len(scaled_mat_orig)),replace=False)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = -1
  
  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs['Cluster'].values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs['Cluster'].values[unlabeled_idx], preds)
  acc_scorePCA100D.append(acc)

 
  # print(f"nnNCA fit in {toc - tic:0.4f} seconds")

In [36]:
print(acc_scorePCA100D)

[0.8901468246948523, 0.8936847691491243, 0.8935668376673153]


PCA 15D

In [37]:

acc_scorePCA15D = []

for i in range(3):

  tsvd = TruncatedSVD(n_components=15)
  x_pca = tsvd.fit_transform(scaled_mat_orig)
  
  labels = np.array([lab4]).copy().astype(np.int8)
  train_inds = np.random.choice(len(scaled_mat_orig), size = int(0.7*len(scaled_mat_orig)),replace=False)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = -1
  
  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs['Cluster'].values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs['Cluster'].values[unlabeled_idx], preds)
  acc_scorePCA15D.append(acc)

 
  # print(f"nnNCA fit in {toc - tic:0.4f} seconds")

In [38]:
print(acc_scorePCA15D)

[0.8929182145173654, 0.8933899404446017, 0.8939206321127425]


### **Save Analysis Output**

In [39]:
vals = pd.DataFrame()

vals['Accuracy'] = acc_score_2D + acc_score_15_2D + acc_scorePCA  +acc_scorePCA100D + acc_scorePCA15D#+ netAE_score + netAE_score2

In [40]:
vals['Embed'] = ['PCA 50D UMAP Sup.','PCA 50D UMAP','PCA 50D t-SNE']*3 + ['PCA 15D UMAP Sup.','PCA 15D UMAP','PCA 15D t-SNE']*3 + ['PCA 50D']*3  + ['PCA 100D']*3 + ['PCA 15D']*3#+ ['netAE']*2

In [41]:
vals['Label'] = ['CellType']*27#+  ['CellType1'] #+  ['Gender2']

In [42]:
vals


,Accuracy,Embed,Label
0,0.888909,PCA 50D UMAP Sup.,CellType
1,0.848871,PCA 50D UMAP,CellType
2,0.885194,PCA 50D t-SNE,CellType
3,0.881007,PCA 50D UMAP Sup.,CellType
4,0.850404,PCA 50D UMAP,CellType
5,0.884250,PCA 50D t-SNE,CellType
6,0.892800,PCA 50D UMAP Sup.,CellType
7,0.859072,PCA 50D UMAP,CellType
8,0.887965,PCA 50D t-SNE,CellType
9,0.874875,PCA 15D UMAP Sup.,CellType


In [43]:
from google.colab import files

vals.to_csv('allUtero105Preds.csv')
files.download('allUtero105Preds.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>